In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..
from fastai.text import *

/home/pczapla/workspace/ulmfit-multilingual


In [3]:
from multifit.datasets import ULMFiTDataset, Dataset
from multifit import ULMFiT, multifit1552_fp16, multifit1552_fp32

# Wikipedia Pretraining

In [4]:
import inspect
print(inspect.getsource(multifit1552_fp16))
print(inspect.getsource(multifit1552_fp32))

def multifit1552_fp16():
    return multifit1552_fp32(bs=128).replace_(fp16=True, name=multifit1552_fp16.__name__)

def multifit1552_fp32(bs=64):
    self = ULMFiT()
    self.replace_(
        label_smoothing_eps=0.0,
        true_wd=True,
        wd=0.1,
        seed=0,
        fp16=False,
        bs=bs,
        use_adam_08=False,
        early_stopping=None,
        name=multifit1552_fp32.__name__
    )
    self.arch.replace_(
        tokenizer='fsp',
        max_vocab=15000,
        qrnn=True,
        n_layers=4,
        n_hid=1552
    )
    self.pretrain_lm.replace_(num_epochs=10, drop_mult=0.5, lr=(1e-2 * bs / 48))
    self.finetuine_lm.replace_(num_epochs=10, drop_mult=1.0, lr=(1e-3 * bs / 48))
    self.classifier.replace_(num_epochs=8, drop_mult=0.5, bs=20, label_smoothing_eps=0.1)
    return self



In [5]:
exp = multifit1552_fp16()

In [6]:
wiki_dataset = exp.arch.dataset(Path(f'data/wiki/ja-100'))

In [7]:
wiki_dataset.load_lm_databunch(bs=128,bptt=70).show_batch()

Data lm-notst, trn: 120037, val: 63
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']


idx,text
0,▁月 ▁9 ▁日 ▁) ▁は ▁、 ▁日 ▁本 ▁の ▁ 漫 ▁画 ▁家 ▁、 ▁ アニメーター ▁、 ▁ アニメーション ▁監 ▁ 督 ▁。 ▁大 ▁ 阪 ▁ 帝 ▁国 ▁大 ▁学 ▁ 附 ▁ 属 ▁ 医 ▁学 ▁ 専 ▁ 門 ▁部 ▁を ▁ 卒 ▁業 ▁、 ▁ 医 ▁ 師 ▁ 免 ▁ 許 ▁取 ▁ 得 ▁、 ▁ のち ▁ 医 ▁学 ▁ 博 ▁ 士 ▁(
1,▁では ▁小 ▁学 ▁校 ▁の ▁教 ▁ 室 ▁が ▁ 不 ▁ 足 ▁するなど ▁、 ▁人 ▁ 口 ▁が ▁ 急 ▁ 増 ▁している ▁。 ▁これらの ことから ▁、 ▁ 老 ▁年 ▁人 ▁ 口 ▁の ▁ 比 ▁ 率 ▁が ▁大 ▁ 阪 ▁市 ▁24 ▁区 ▁の ▁中 ▁で ▁最 ▁も ▁ 低 ▁くなっている ▁。 ▁地 ▁理 ▁. ▁区 ▁ 域 ▁は ▁東 ▁を ▁西 ▁ 横 ▁ 堀 ▁川
2,トラック などに ▁手 ▁ 荷 ▁物 ▁が ▁ 残 ▁ っ ていても それが ▁ 忘 ▁れ ▁物 ▁である とは ▁ 認 ▁ 識 ▁しない ▁ 可 ▁ 能 ▁性 ▁が ▁高 ▁い ▁。 ▁また ▁ 忘 ▁れ ▁物 ▁として ▁手 ▁ 荷 ▁物 ▁を ▁発 ▁見 ▁し ていても ▁、 ▁それが ▁ 爆 ▁発 ▁物 ▁である ▁事 ▁には ▁ 気 ▁ 付 ▁かない ▁ 可 ▁ 能 ▁性 ▁もあるが ▁、 ▁
3,郎 ▁は ▁ 淀 ▁野 ▁と ▁ 近 ▁所 ▁を ▁ 散 ▁ 歩 ▁中 ▁、 ▁「 ▁東 ▁京 ▁の ▁ 横 ▁ 光 ▁は どう や ▁? ▁」 ▁と ▁ 質 ▁ 問 ▁し ▁、 ▁ 勢 ▁いの あった ▁ 横 ▁ 光 ▁ 利 ▁一 ▁を ライバル ▁ 視 ▁していた ▁。 ▁12 ▁月 ▁下 ▁ 旬 ▁、 ▁ 母 ▁が ▁ 阿 ▁ 倍 ▁野 ▁の ▁小 ▁間
4,▁ 速 ▁に アラブ ▁化 ▁・ ▁ イスラム ▁化 ▁した ▁。 ▁8 ▁世 ▁ 紀 ▁には アッ バース ▁ 朝 ▁の カリフ が バグ ダー ド に ▁ 都 ▁を ▁ 造 ▁ 営 ▁し ▁、 ▁ アッ バース ▁ 朝 ▁が ▁ 滅 ▁びる まで イスラム ▁世 ▁ 界 ▁の ▁ 精 ▁神 ▁的 ▁中 ▁ 心 ▁として ▁ 栄 ▁えた ▁。 ▁10 ▁世 ▁ 紀 ▁ 末 ▁に


In [10]:
exp.pretrain_lm.train_(wiki_dataset)

Setting LM weights seed seed to 0
Training args:  {'drop_mult': 0.5, 'true_wd': True, 'wd': 0.1, 'pretrained': False} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.1, 'hidden_p': 0.15, 'input_p': 0.25, 'embed_p': 0.02, 'weight_p': 0.2, 'tie_weights': True, 'out_bias': True}
Data lm-notst, trn: 120037, val: 63
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Setting LM training seed seed to 0
Experiment data/wiki/ja-100/models/fsp15k/multifit1552_fp16
Training lm from random weights


epoch,train_loss,valid_loss,accuracy,time
0,3.107674,3.097375,0.411802,57:20
1,3.331074,3.364228,0.382917,57:24
2,4.564233,4.470573,0.283381,57:10
3,3.628411,3.552323,0.364708,56:55
4,3.489929,3.459749,0.370721,56:50
5,3.310685,3.333506,0.383702,56:54
6,3.154821,3.179965,0.401726,57:02
7,2.972626,2.931468,0.432523,57:05
8,2.831486,2.714095,0.461959,57:06
9,2.760642,2.598436,0.481062,57:07


this Learner object self-destroyed - it still exists, but no longer usable
Language model saved to data/wiki/ja-100/models/fsp15k/multifit1552_fp16
Saving dump to data/wiki/ja-100/models/fsp15k/multifit1552_fp16/pretraining.json


# MLDoc Trainig

In [7]:
mldoc_dataset = exp.arch.dataset(Path('data/mldoc/ja-1'))

In [8]:
mldoc_dataset.load_clas_databunch(bs=128)[0].show_batch()

Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000


text,target
▁xxbos ▁xxfld ▁1 ▁ [ ▁東 ▁京 ▁25 ▁日 ▁ ロイ ター ▁ ] ▁野 ▁ 村 ▁ 総 ▁ 研 ▁が ▁、 ▁9 ▁月 ▁24 ▁日 ▁ 付 ▁で まとめた ▁ 香 ▁ 港 ▁・ ▁ シン ガ ポー ▁ ル ▁・ ▁ マレーシア ▁市 ▁場 ▁の ▁主 ▁要 ▁株 ▁式 ▁の ▁ 投 ▁ 資 ▁ 判 ▁ 断 ▁は ▁、 ▁以 ▁下 ▁の ▁通 ▁り ▁。 ▁(,0
▁xxbos ▁xxfld ▁1 ▁ [ ▁東 ▁京 ▁25 ▁日 ▁ ロイ ター ▁ ] ▁野 ▁ 村 ▁ 総 ▁ 研 ▁が ▁、 ▁11 ▁月 ▁25 ▁日 ▁ 付 ▁で まとめた ▁ 香 ▁ 港 ▁・ ▁ シン ガ ポ ▁ ール ▁・ ▁ マレーシア ▁市 ▁場 ▁の ▁主 ▁要 ▁株 ▁式 ▁の ▁ 投 ▁ 資 ▁ 判 ▁ 断 ▁は ▁、 ▁以 ▁下 ▁の ▁通 ▁り ▁。 ▁(,0
▁xxbos ▁xxfld ▁1 ▁ 直 ▁ 近 ▁上 ▁ 値 ▁ 抵 ▁ 抗 ▁ 短 ▁期 ▁上 ▁ 値 ▁ 抵 ▁ 抗 ▁ 直 ▁ 近 ▁下 ▁ 値 ▁ 支 ▁ 持 ▁ 短 ▁期 ▁下 ▁ 値 ▁ 支 ▁ 持 ▁ ドル ▁ / ▁ 円 ▁ 125 ▁. ▁ 60 ▁12 6 ▁. ▁00 ▁ 124 ▁. ▁ 60 ▁ 124 ▁. ▁ 40,3
▁xxbos ▁xxfld ▁1 ▁ [ ▁東 ▁京 ▁26 ▁日 ▁ ロイ ター ▁ ] ▁ 今 ▁ 週 ▁は ▁、 ▁28 ▁日 ▁発 ▁表 ▁の 8 ▁月 ▁ 調 ▁ 査 ▁日 ▁ 銀 ▁ 短 ▁ 観 ▁( ▁ 企 ▁業 ▁ 短 ▁期 ▁経 ▁ 済 ▁ 観 ▁ 測 ▁ 調 ▁ 査 ▁) ▁、 ▁30 ▁日 ▁に ▁発 ▁表 ▁される 7 ▁月 ▁ 鉱 ▁ 工,3
▁xxbos ▁xxfld ▁1 ▁ レート は ▁ 終 ▁ 値 ▁( ▁前 ▁日 ▁ 比 ▁または ▁前 ▁ 週 ▁ 末 ▁ 比 ▁) ▁、 ▁ 安 ▁ 値 ▁ 〜 ▁高 ▁ 値 ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━ ▁ ━,3


In [12]:
exp.finetune_lm.train_(mldoc_dataset)

Setting LM weights seed seed to 0
Training args:  {'drop_mult': 1.0, 'true_wd': True, 'wd': 0.1, 'pretrained': True, 'pretrained_fnames': [PosixPath('/home/pczapla/workspace/ulmfit-multilingual/data/wiki/ja-100/models/fsp15k/multifit1552_fp16/lm_best'), PosixPath('/home/pczapla/workspace/ulmfit-multilingual/data/wiki/ja-100/models/fsp15k/itos')]} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.1, 'hidden_p': 0.15, 'input_p': 0.25, 'embed_p': 0.02, 'weight_p': 0.2, 'tie_weights': True, 'out_bias': True}
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Setting LM training seed seed to 0
Experiment data/mldoc/ja-1/models/fsp15k/multifit1552_fp16
Fitting using 2 cycle fit schedule


epoch,train_loss,valid_loss,accuracy,time
0,2.219860,1.833544,0.606933,01:26


epoch,train_loss,valid_loss,accuracy,time
0,1.944596,1.670773,0.637072,01:49
1,1.806380,1.528684,0.662753,01:49
2,1.720533,1.444982,0.677305,01:49
3,1.663399,1.379856,0.688376,01:49
4,1.618090,1.334722,0.696019,01:49
5,1.572266,1.292020,0.704291,01:49
6,1.550954,1.254581,0.711302,01:49
7,1.474405,1.226887,0.716287,01:49
8,1.446200,1.212317,0.718968,01:49
9,1.445466,1.209516,0.719546,01:50


this Learner object self-destroyed - it still exists, but no longer usable
Language model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/finetuining.json


In [9]:
exp.load_(Path('data/mldoc/ja-1/models/fsp15k/multifit1552_fp16')).finetune_lm

Loading data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/finetuining.json
Loading data/wiki/ja-100/models/fsp15k/multifit1552_fp16/pretraining.json


ULMFiTFinetuining(seed=0, name='multifit1552_fp16', experiment_path=PosixPath('data/mldoc/ja-1/models/fsp15k/multifit1552_fp16'), dataset_path=PosixPath('data/mldoc/ja-1'), num_epochs=10, bs=128, bptt=70, drop_mult=1.0, label_smoothing_eps=0.0, use_adam_08=False, true_wd=True, wd=0.1, fp16=True, lr=0.0026666666666666666, pretrained=True)

In [10]:
exp.classifier.train_(seed=0)

Setting Classifier weights seed seed to 0
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 0


epoch,train_loss,valid_loss,accuracy,time
0,0.911967,0.639816,0.850000,00:14
1,0.781325,0.646144,0.874000,00:13
2,0.703413,0.722381,0.849000,00:13
3,0.622030,0.678657,0.875000,00:13
4,0.545690,0.623689,0.890000,00:13
5,0.491453,0.628224,0.883000,00:14
6,0.451803,0.628584,0.886000,00:14
7,0.429028,0.627261,0.888000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [11]:
exp.classifier.train_(seed=1)

Setting Classifier weights seed seed to 1
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 1


epoch,train_loss,valid_loss,accuracy,time
0,0.908408,0.654319,0.860000,00:14
1,0.790398,0.768438,0.812000,00:14
2,0.692833,0.650352,0.883000,00:14
3,0.607950,0.659773,0.867000,00:14
4,0.549019,0.627178,0.886000,00:14
5,0.497783,0.701499,0.879000,00:14
6,0.455803,0.651905,0.870000,00:14
7,0.430048,0.634729,0.879000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed1
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed1/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [12]:
exp.classifier.train_(seed=2)

Setting Classifier weights seed seed to 2
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 2


epoch,train_loss,valid_loss,accuracy,time
0,0.925113,0.653345,0.849000,00:14
1,0.774887,0.632942,0.876000,00:14
2,0.686229,0.661485,0.870000,00:14
3,0.630340,0.630609,0.877000,00:14
4,0.550789,0.618177,0.878000,00:14
5,0.494387,0.636892,0.878000,00:14
6,0.450500,0.633069,0.885000,00:14
7,0.431031,0.637073,0.881000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed2
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed2/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [13]:
exp.classifier.train_(seed=3)

Setting Classifier weights seed seed to 3
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 3


epoch,train_loss,valid_loss,accuracy,time
0,0.903641,0.616180,0.872000,00:14
1,0.798058,0.611768,0.882000,00:14
2,0.692193,0.674029,0.842000,00:14
3,0.622152,0.605841,0.889000,00:14
4,0.545950,0.640584,0.877000,00:14
5,0.495717,0.625205,0.885000,00:14
6,0.454127,0.629366,0.885000,00:14
7,0.430954,0.627262,0.891000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed3
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed3/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [14]:
exp.classifier.train_(seed=4)

Setting Classifier weights seed seed to 4
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 4


epoch,train_loss,valid_loss,accuracy,time
0,0.885045,0.623022,0.863000,00:14
1,0.791488,0.675674,0.850000,00:14
2,0.702259,0.796426,0.834000,00:14
3,0.626244,0.681571,0.865000,00:14
4,0.558977,0.633119,0.890000,00:14
5,0.493925,0.646156,0.875000,00:14
6,0.450324,0.630983,0.885000,00:14
7,0.428313,0.649953,0.879000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed4
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed4/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [15]:
exp.classifier.train_(seed=5)

Setting Classifier weights seed seed to 5
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 5


epoch,train_loss,valid_loss,accuracy,time
0,0.898696,0.634723,0.865000,00:14
1,0.787859,0.628963,0.878000,00:14
2,0.700993,0.642409,0.855000,00:14
3,0.619737,0.632098,0.880000,00:14
4,0.558749,0.729376,0.861000,00:14
5,0.489555,0.629867,0.877000,00:14
6,0.451724,0.644491,0.876000,00:14
7,0.429537,0.651178,0.879000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed5
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed5/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [16]:
exp.classifier.train_(seed=6)

Setting Classifier weights seed seed to 6
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 6


epoch,train_loss,valid_loss,accuracy,time
0,0.891280,0.656561,0.861000,00:14
1,0.771049,0.648831,0.859000,00:14
2,0.677718,0.681419,0.859000,00:14
3,0.622475,0.635418,0.872000,00:14
4,0.555276,0.639490,0.875000,00:14
5,0.492825,0.653272,0.873000,00:14
6,0.449347,0.649654,0.873000,00:14
7,0.425822,0.639283,0.874000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed6
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed6/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


In [17]:
exp.classifier.train_(seed=7)

Setting Classifier weights seed seed to 7
Data lm-notst, trn: 9900, val: 1100
Size of vocabulary: 15000
First 20 words in vocab: ['▁xxunk', '▁xxpad', '▁xxbos', '▁xxeos', '▁xxfld', '▁xxmaj', '▁xxup', '▁xxrep', '▁xxwrep', '<unk>', '▁', '▁、', '▁。', '▁の', '▁に', '▁を', '▁年', '▁は', '▁・', '▁(']
Data cls, trn: 1000, val: 1000
Data tst, trn: 1000, val: 4000
Training args:  {'drop_mult': 0.5, 'wd': 0.1, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy()} config:  {'emb_sz': 400, 'n_hid': 1552, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.4, 'hidden_p': 0.3, 'input_p': 0.4, 'embed_p': 0.05, 'weight_p': 0.5}
Loading pretrained model /home/pczapla/workspace/ulmfit-multilingual/data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/enc_best
Setting Classifier training seed seed to 7


epoch,train_loss,valid_loss,accuracy,time
0,0.890206,0.618165,0.869000,00:14
1,0.787146,0.614832,0.884000,00:14
2,0.695766,0.705636,0.843000,00:14
3,0.625174,0.611888,0.890000,00:14
4,0.561164,0.677992,0.856000,00:14
5,0.496124,0.623592,0.881000,00:14
6,0.454188,0.647716,0.876000,00:14
7,0.431855,0.633412,0.879000,00:14


Classifier model saved to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed7
Saving dump to data/mldoc/ja-1/models/fsp15k/multifit1552_fp16seed7/classifier.json
this Learner object self-destroyed - it still exists, but no longer usable


# Results

In [9]:
def get_results(exp_path):
    exp = ULMFiT().load_(exp_path).classifier
    results = exp.validate() 
    results.update(seed=exp.seed, fp16=exp.fp16)
    return results
results = [get_results(exp_path) for exp_path in Path('data/mldoc/ja-1/models/fsp15k').glob("multifit1552_fp16*")]
results_df = pd.DataFrame.from_records(results)
results_df.sort_values(["valid accuracy"])[["name", "seed", "test accuracy", "valid accuracy"]]

{'test loss': 0.5893779397010803, 'test FBeta': 0.9026554822921753, 'test Precision': 0.9043030738830566, 'test Recall': 0.902697741985321, 'test accuracy': 0.903249979019165, 'name': 'multifit1552_fp16seed1', 'valid loss': 0.6347294449806213, 'valid FBeta': 0.8822355270385742, 'valid Precision': 0.8312159776687622, 'valid Recall': 0.8805031180381775, 'valid accuracy': 0.9299362897872925, 'train loss': 0.3954845070838928, 'train FBeta': 0.9960629940032959, 'train Precision': 0.9978118538856506, 'train Recall': 1.0, 'train accuracy': 0.9980732202529907}
Loading data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/classifier.json
Loading data/mldoc/ja-1/models/fsp15k/multifit1552_fp16/finetuining.json
Loading data/wiki/ja-100/models/fsp15k/multifit1552_fp16/pretraining.json


,name,seed,test accuracy,valid accuracy
5,multifit1552_fp16seed4,4,0.89675,0.918580
1,multifit1552_fp16seed6,6,0.89800,0.921776
3,multifit1552_fp16seed5,5,0.89775,0.923404
2,multifit1552_fp16seed7,7,0.90200,0.927660
0,multifit1552_fp16seed2,2,0.90275,0.929336
6,multifit1552_fp16seed1,1,0.90325,0.929936
7,multifit1552_fp16,0,0.89950,0.936170
4,multifit1552_fp16seed3,3,0.90250,0.937634
